## Vitess

![](https://vitess.io/img/logos/vitess-stacked.png)

Quelle: https://vitess.io/

- - -

Vitess ist eine Datenbanklösung für die Bereitstellung, Skalierung und Verwaltung großer Cluster von Open-Source-Datenbankinstanzen. Sie unterstützt aktuell MySQL und Percona Server für MySQL. Sie ist so konzipiert, dass sie in öffentlichen oder privaten Cloud-Architekturen genauso effizient läuft wie auf dedizierter Hardware. Sie kombiniert und erweitert viele wichtige SQL-Funktionen mit der Skalierbarkeit einer NoSQL-Datenbank. 

Vitess kann Sie bei folgenden Problemen unterstützen:
* Skalieren Sie eine SQL-Datenbank, indem Sie sie in Sharding aufteilen und gleichzeitig die Anwendungsänderungen auf ein Minimum beschränken.
* Migration von Bare-Metal oder VMs in eine private oder öffentliche Cloud.
* Bereitstellen und Verwalten einer großen Anzahl von SQL-Datenbankinstanzen.

Vitess enthält kompatible JDBC- und Go-Datenbanktreiber mit nativem Abfrageprotokoll. Darüber hinaus implementiert es das MySQL-Serverprotokoll, das mit praktisch jeder anderen Sprache kompatibel ist.

Vitess hat über fünf Jahre lang den gesamten YouTube-Datenbankverkehr abgewickelt. Viele Unternehmen nutzen Vitess mittlerweile für ihre Produktionsanforderungen.

Quelle: https://vitess.io/docs/21.0/overview/whatisvitess/
- - -

Zuerst holen wir uns die vitess Distribution und stellen sie lokal im Verzeichnis `/usr/local/vitess` zur Verfügung.


In [ ]:
%%bash
version="v21.0"
url="$(curl -s https://api.github.com/repos/vitessio/vitess/releases | jq --arg version "${version}" -r '[.[] | select(.tag_name | contains($version))] | sort_by(.created_at) | reverse | .[0:1] | .[] | .assets[] | select(.content_type | contains("application/gzip")) | .browser_download_url')"
file="${url##*/}"
curl -LO "${url}"

tar -xzf ${file}
cd ${file/.tar.gz/}
sudo mkdir -p /usr/local/vitess
sudo cp -r * /usr/local/vitess/

Von Mysql brauchen das CLI `mysql`

In [ ]:
%%bash
sudo apt-get install -y mysql-client

Dann folgt die Installation des vitess Operators

In [ ]:
%%bash
cd /usr/local/vitess/examples/operator
kubectl apply -f operator.yaml

Im Installations-Verzeichnis `/usr/local/vitess/` finden Sie eine Gruppe von YAML-Dateien. 

Die erste Ziffer jedes Dateinamens gibt die Phase des Beispiels an. 

Die nächsten beiden Ziffern geben die Reihenfolge an, in der sie ausgeführt werden. Beispielsweise `101_initial_cluster.yaml`  die erste Datei der ersten Phase. 

In [ ]:
%%bash
ls -l /usr/local/vitess/examples/operator

Wir bauen uns einen ersten Example Cluster

In [ ]:
%%bash
cd /usr/local/vitess/examples/operator
kubectl apply -f 101_initial_cluster.yaml
kubectl get pods

Wir machen die Ports der wichtigsten Services gehen aussen verfügbar

In [ ]:
%%bash
export VTCTLD_SERVICE=$(kubectl get service --selector="planetscale.com/component=vtctld" -o name | head -n1)
kubectl patch "$VTCTLD_SERVICE" --type='merge' -p '{
"spec": {
  "type": "NodePort"
}
}'
export VTGATE_SERVICE=$(kubectl get service --selector='planetscale.com/component=vtgate,!planetscale.com/cell' -o name | head -n1)
kubectl patch "$VTGATE_SERVICE" --type='merge' -p '{
"spec": {
  "type": "NodePort"
}
}'
export VTADMIN_SERVICE=$(kubectl get service --selector="planetscale.com/component=vtadmin" -o name | head -n1)
kubectl patch "$VTADMIN_SERVICE" --type='merge' -p '{
"spec": {
  "type": "NodePort"
}
}'

# Abrufen der Server-IP 
SERVER_IP=$(cat ~/work/server-ip)

# Funktion zum Extrahieren der zugewiesenen NodePorts
get_node_ports() {
  kubectl get "$1" -o jsonpath='{range .spec.ports[*]}{.name}{" "}{.nodePort}{"\n"}{end}'
}

get_node_ports "$VTCTLD_SERVICE" | while read name port; do
  echo "http://$SERVER_IP:$port "$VTCTLD_SERVICE" ($name)"
done
echo ""

get_node_ports "$VTGATE_SERVICE" | while read name port; do
  echo "http://$SERVER_IP:$port "$VTGATE_SERVICE" ($name)"
done
echo ""

get_node_ports "$VTADMIN_SERVICE" | while read name port; do
  echo "http://$SERVER_IP:$port "$VTADMIN_SERVICE" ($name)"
done
echo ""

### Create Schema 
Load our initial schema:

In [ ]:
%%bash
export VTCTLD_SERVICE=$(kubectl get service --selector="planetscale.com/component=vtctld" -o name | head -n1)
export VTCTLD_PORT=$(kubectl get $VTCTLD_SERVICE -o jsonpath='{range .spec.ports[1]}{.nodePort}{"\n"}{end}')

/usr/local/vitess/bin/vtctldclient ApplySchema --sql-file="/usr/local/vitess/examples/operator/create_commerce_schema.sql" \
    commerce --server=localhost:$VTCTLD_PORT

/usr/local/vitess/bin/vtctldclient ApplyVSchema \
    --vschema-file="/usr/local/vitess/examples/operator/vschema_commerce_initial.json" commerce \
    --server=localhost:$VTCTLD_PORT


Wir schreiben ein wenig Daten:

In [ ]:
%%bash
cat <<EOF >insert_data.sql
USE commerce;

-- Produkt-Tabelle mit zufälligen Daten füllen
INSERT INTO product (sku, description, price) VALUES
  (UUID(), 'Laptop', 120000),
  (UUID(), 'Smartphone', 80000),
  (UUID(), 'Tablet', 50000),
  (UUID(), 'Monitor', 30000),
  (UUID(), 'Headphones', 15000);

INSERT INTO customer (email) VALUES
  (UUID()),
  (UUID()),
  (UUID()),
  (UUID()),
  (UUID());

-- Bestellungen mit zufälligen Kunden und Produkten füllen
INSERT INTO corder (customer_id, sku, price)
SELECT 
  (SELECT customer_id FROM customer ORDER BY RAND() LIMIT 1),
  (SELECT sku FROM product ORDER BY RAND() LIMIT 1),
  (SELECT price FROM product ORDER BY RAND() LIMIT 1)
FROM (SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5) t;
EOF


In [ ]:
%%bash
export VTGATE_SERVICE=$(kubectl get service --selector='planetscale.com/component=vtgate,!planetscale.com/cell' -o name | head -n1)
export VTGATE_PORT=$(kubectl get $VTGATE_SERVICE -o jsonpath='{range .spec.ports[2]}{.nodePort}{"\n"}{end}')

mysql -h 127.0.0.1 -P $VTGATE_PORT -u user -e "show databases;"
mysql -h 127.0.0.1 -P $VTGATE_PORT -u user <insert_data.sql
mysql -h 127.0.0.1 -P $VTGATE_PORT -u user -e 'use commerce; select * from product; select * from customer'


Mittels MySQL CLI verbinden.

Dazu zuerst in die VM einloggen

In [ ]:
%%bash
export VTGATE_SERVICE=$(kubectl get service --selector='planetscale.com/component=vtgate,!planetscale.com/cell' -o name | head -n1)
export VTGATE_PORT=$(kubectl get $VTGATE_SERVICE -o jsonpath='{range .spec.ports[2]}{.nodePort}{"\n"}{end}')

echo "ssh -i ~/.ssh/lerncloud ubuntu@$(cat ~/work/server-ip)"
echo "mysql -h 127.0.0.1 -P $VTGATE_PORT -u user"

Weiter geht es mit [Move Tables](https://vitess.io/docs/21.0/user-guides/migration/move-tables/)

### Aufräumen


In [ ]:
%%bash
cd /usr/local/vitess/examples/operator
kubectl delete -f 101_initial_cluster.yaml
kubectl delete -f operator.yaml